In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
train_x = pd.read_csv('data_train_x.csv')
train_y = pd.read_csv('data_train_y.csv')
valid_x = pd.read_csv('data_valid_x.csv')
valid_y = pd.read_csv('data_valid_y.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (38,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (24,35,36,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
valid_y.shape

(10070, 1)

In [11]:
train_x.columns

Index(['購物車編號', '主單編號', '訂單編號', '轉單日期時間', '預計出貨日期', '配送方式', '通路商', '溫層類別',
       '收件人', '收件人電話', '地址_x', '門市', '訂單來源', '商品名稱', '商品選項', '商品料號', '數量',
       '商品單價', '運費', '配送編號', '狀態日期', '出貨單狀態', '訂單狀態', '活動代碼', '活動名稱', '折扣金額',
       '銷售金額(折扣後)', '付款方式', '活動折扣金額', '折價券活動序號', '折價券活動名稱', '折價券折扣金額',
       '貨到物流中心日', '建議貨到期限', '會員編號', '商店備註', '訂購備註', '貨運單備註', '驗退原因說明',
       '訂單確認日期', '實體會員編號', '商品屬性', '商品贈品關聯代碼', '購買人', '購買人會員等級', '活動對象',
       '活動會員等級', '總成本', '是否為加價購品', '國碼', '收件國家', '取消原因', '購物車總額', '商品頁序號',
       '點數活動名稱', '折抵點數', '點數折扣金額', 'eCOM會員編號', '手機號碼', '廠商自訂會員編號1',
       '廠商自訂會員編號2', '會員卡名稱', '全名', '性別', '生日', 'Email', '最後消費日期', '最高消費門市名稱',
       '最高消費來源', '最近一次消費門市名稱', '最近一次消費來源', '等級內累積消費金額', '總累積消費金額', '註冊來源',
       '註冊門市', '註冊門市店櫃代號', '會員卡起始日', '會員卡到期日', '國家', '城市', '區', '地址_y',
       '國家/地區'],
      dtype='object')

訂單確認日期 有nan
商品屬性 nan, '活動贈品', '商品贈品'
通路商 '7-11(統一)', '全家', nan
商品贈品關聯代碼 nan應該是沒贈品吧?
購買人 可能為nan
商品頁序號 雖然都是數字 不過應該當類別處理
折抵點數 = 點數折扣金額 * -1
eCOM會員編號 = 會員編號
購買人會員等級 = 會員卡名稱
購買人 = 全名
性別 有nan
生日 有nan
最高消費來源 有nan
最近一次消費來源 有nan
區 有nan
地址_y 有nan
國碼 和 收件國家 分別對應 ['台灣', '馬來西亞', '香港'] 可是 國家/地區 除了前述三者 還多了 日本

In [88]:
train_x['收件國家'].unique()

array(['台灣', '馬來西亞', '香港'], dtype=object)

In [89]:
del_col_name = ['溫層類別', '商店備註', '訂購備註', '貨運單備註', '驗退原因說明', '實體會員編號'
                ,'活動對象', '活動會員等級', '取消原因', '點數活動名稱', '折抵點數'
               ,'eCOM會員編號', '廠商自訂會員編號1', '廠商自訂會員編號2', '會員卡名稱'
               ,'全名', '最高消費門市名稱', '最近一次消費門市名稱', '註冊門市', '註冊門市店櫃代號'
               ,'國家', '城市', '區', '地址_y']
train_x.drop(del_col_name,axis=1, inplace=True)
valid_x.drop(del_col_name,axis=1, inplace=True)

In [91]:
train_x.columns

Index(['購物車編號', '主單編號', '訂單編號', '轉單日期時間', '預計出貨日期', '配送方式', '通路商', '收件人',
       '收件人電話', '地址_x', '門市', '訂單來源', '商品名稱', '商品選項', '商品料號', '數量', '商品單價',
       '運費', '配送編號', '狀態日期', '出貨單狀態', '訂單狀態', '活動代碼', '活動名稱', '折扣金額',
       '銷售金額(折扣後)', '付款方式', '活動折扣金額', '折價券活動序號', '折價券活動名稱', '折價券折扣金額',
       '貨到物流中心日', '建議貨到期限', '會員編號', '訂單確認日期', '商品屬性', '商品贈品關聯代碼', '購買人',
       '購買人會員等級', '總成本', '是否為加價購品', '國碼', '收件國家', '購物車總額', '商品頁序號', '點數折扣金額',
       '手機號碼', '性別', '生日', 'Email', '最後消費日期', '最高消費來源', '最近一次消費來源',
       '等級內累積消費金額', '總累積消費金額', '註冊來源', '會員卡起始日', '會員卡到期日', '國家/地區'],
      dtype='object')

In [97]:
valid_x.columns

Index(['購物車編號', '主單編號', '訂單編號', '轉單日期時間', '預計出貨日期', '配送方式', '通路商', '收件人',
       '收件人電話', '地址_x', '門市', '訂單來源', '商品名稱', '商品選項', '商品料號', '數量', '商品單價',
       '運費', '配送編號', '狀態日期', '出貨單狀態', '訂單狀態', '活動代碼', '活動名稱', '折扣金額',
       '銷售金額(折扣後)', '付款方式', '活動折扣金額', '折價券活動序號', '折價券活動名稱', '折價券折扣金額',
       '貨到物流中心日', '建議貨到期限', '會員編號', '訂單確認日期', '商品屬性', '商品贈品關聯代碼', '購買人',
       '購買人會員等級', '總成本', '是否為加價購品', '國碼', '收件國家', '購物車總額', '商品頁序號', '點數折扣金額',
       '手機號碼', '性別', '生日', 'Email', '最後消費日期', '最高消費來源', '最近一次消費來源',
       '等級內累積消費金額', '總累積消費金額', '註冊來源', '會員卡起始日', '會員卡到期日', '國家/地區'],
      dtype='object')

In [106]:
pred_colum = ['數量', '商品單價', '運費', '折扣金額', '銷售金額(折扣後)', '活動折扣金額', '折價券活動序號', '折價券折扣金額',
       '總成本', '購物車總額', '商品頁序號', '點數折扣金額', '等級內累積消費金額', '總累積消費金額']

In [107]:
model = XGBClassifier()
model.fit(train_x[pred_colum], train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [109]:
y_pred = model.predict(valid_x[pred_colum])
predictions = [round(value) for value in y_pred]

In [110]:
# evaluate predictions
accuracy = accuracy_score(valid_y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 98.22%
